# Introduction

# Dataset
The dataset provided by KKBOX and the 11th ACM International Conference on Web Search and Data Mining (WSDM 2018) is used in our task. The dataset includes user information, song information and users' listening events, which records whether the user has listened to this song or not. Since we don't have groundtruth for their provided testing dataset, we split the data into three parts: 80% for training, 10% for validation and 10% for testing. After data preprocessing and filtering out inappropriate samples, we get 34288 users and 

# Task - Listen prediction
In this task, we would like to predict whether the user will listen to the song or not.  
## Baseline
The baseline method returns True if the song is popular, using a threshold of the 50% of popularity. That is, as long as the count of the song is larger than $\frac{TotalCount}{2}$, it will be predicted as a positive sample. This method gets the accuracy 47.96%, which is even lower than random guess (50%). We infer that it doesn't take the relationship between certain users and songs into consideration, so the model is unable to correctly predict based on the given information.  

## Jaccard
To dig further into our dataset, we further use the similarity to understand and measure the relationship between users and songs. Jaccard similarity emphasizes similarity between finite sample sets, and is formally defined as the size of the intersection divided by the size of the union of the sample sets. The mathemetical representation is shown as below. 
  
\begin{equation*}
J(A,B) = \frac{|A|\cap|B|}{|A|\cup|B|}
\end{equation*}
  
Given a pair (u, s) in the testing set, consider all training items s' that user u has listened. For each, compute the Jaccard similarity between s and s' , i.e., users (in the training set) who have listen to s and users who have listen to s'. Predict as True if the maximum of these Jaccard similarities exceeds a certain threshold.  

## One-class Recommendation
In this section, we introduce the concept of latent factor model that there exists an unknown low-dimensional representation (latent factor) of users and items where user-item affinity can be modeled accurately. Thus, we try to build a model elates a set of observable variables to a set of latent variables. And the difference between latent factor model and one-class recommendation is to predict binary outcomes, instead of predicting real values. Suppose we have binary observations R (a matrix that records positive and negative feedback for all (u, s) pairs). Then, we try to maximize the difference in predictions between positive and negative items:  
  
\begin{align*}
&Likelihood:maxln\sigma(\gamma_u \cdot \gamma_i - \gamma_u \cdot \gamma_i)  \\
&Loss=minln(1+e^(\gamma_u\gamma_j-\gamma_u\gamma_i))  \\
\end{align*}
  
for a user $u$ who likes an item $i$ and dislikes an item $j$.  
After optimization, we can get gradient of loss with repect to each parameter.  
  
\begin{align*}
&\frac{\partial L}{\partial \gamma_u}=\frac{(\gamma_j-\gamma_i)e^(\gamma_u(\gamma_j-\gamma_i))}{1+e^(\gamma_u(\gamma_j-\gamma_i))} + \lambda \gamma_u  \\
&\frac{\partial L}{\partial \gamma_i}=\frac{-\gamma_u e^(\gamma_u(\gamma_j-\gamma_i))}{1+e^(\gamma_u(\gamma_j-\gamma_i))} + \lambda \gamma_i  \\
&\frac{\partial L}{\partial \gamma_j}=\frac{\gamma_u e^(\gamma_u(\gamma_j-\gamma_i))}{1+e^(\gamma_u(\gamma_j-\gamma_i))} + \lambda \gamma_j  \\
\end{align*}  
Then, we use stochastic gradient descent to update the model. The learning rate is set as 0.1 and the regularization coefficient $\lambda$ is also 0.1. We have tried the dimension of latent factors from 1 to 3, and the result is shown as below.  

| # of dimension | Accuracy |
|---|---|
|     1     |  54.55%  |
|     2     |  55.21%  |
|     3     |  51.80%  |
  
We found that when the dimension of latent factor is 2, it recieves the highest accuracy 55.21%. And when the dimension keeps increaseing, the accuracy drops dramatically, which indicates that a small number of dimension, like 2, is sufficient enough to represent the features for users and songs.

## Ensemble learning
To further improve the performance of our model, we try to use ensemble learning. Multiple models are strategically generated and combined to solve the problem. In this case, we take the three model we trained in one-class recommendation, and we simply take the mean of the three predicted results to determine whether the sample is positive or negative. We eventually get the highest accuracy with ensemble learning among all methods, which is 60.14%


## Conclusion
In this task, we propose one baseline and three methods, which are Jaccard similarity, one-class recommendation and ensemeble learning respectively. The following is the table that shows their preformance.  
  
| Method | Accuracy |
|---|---|
|  Baseline |  47.96%  |
|   Jaccard |  58.49%  |
| One-class |  55.21%  |
|  Ensemble |  60.14%  |
  
We can see that the baseline only has the accuracy 47.96%, since it does not consider the relationship between users and items. Thus, we try to use other approaches to take it into consideration. For Jaccard similarity, we improve about __ from our baseline. Moreover, one-class recommendatoin introduces the concept of dimensionality reduction and results in a better preformance. Finally, we apply ensemble learning to the task, and reaches 60.14% accuracy. It takes advantage of latent factor and increases the capacity of the models on this task, and eventually gets the best preformance among our proposed methods.